Surface Fuel Updates.. buckle up

There will be 3 GridFire runs representing Year 5, Year 10, and Year 20.

We have 4 year intervals in our ee_treatment simulation images, the pixel values and their Year Interval correspond:

1 (years 1-5) 
2 (years 6-10) 
3 (years 11-15)
4 (years 16-20)

We want to generate newly updated FM40 layers for each GridFire run year. 
Since there is a mismatch in the cadence of GridFire run year intervals and our ee_treatments() year intervals,
we will be updating only those treated pixels with value 1, 2, and 4 from our ee_treatment() images.

For each of those pixel values (1,2,4) we will create a new updated FM40 image. The workflow is this:
* mask to only pixels with that given value, 
* use a simplified lookup table .csv on Cloud Storage (containing from_codes and to_codes) to remap the baseline FM40 values to their updated values in those treated pixels
* then fill in the 'untreated pixels' with the baseline FM40 image

Expected Output is three Updated FM40 images per RunID

Example: RUNID1 = SN WUI 500k
Outputs:
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun1 (updated in ee_treatment pixels==1)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun2 (updated in ee_treatment pixels==2)
* GEE Asset - projects/mas-gee/assets/treatment_scenarios/RunID1/FM40_GFrun3 (updated in ee_treatment pixels==4)

We also are exporting them to Google Drive in case there is a need
* Google Drive - RunDID1_FM40_GFrun1.tif
* Google Drive - RunDID1_FM40_GFrun2.tif
* Google Drive - RunDID1_FM40_GFrun3.tif


In [22]:
import ee
import os
import math
from datetime import datetime
from src.utils import ee_treatments
from src.utils import runs
from src.utils.check_exists import check_exists
from src.utils.yml_params import get_export_params
from src.utils.ee_csv_parser import parse_txt, to_numeric
yml_file = os.path.join(os.getcwd(),'config.yml')
CRS,EXPORT_SCALE = get_export_params(yml_file)

ee.Initialize(project='mas-gee')
%load_ext autoreload
%autoreload 2

# In development phase
tag = 'fm40_updates'
ee.data.setWorkloadTag(tag)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INPUT YOUR TREATED IMAGE PATH BELOW, THEN RUN ALL

In [23]:
# Plug in treatment scenario ee.Image path
treated_path = "projects/mas-gee/assets/treatment_scenarios/RunID33/CC_Fire_1m_20231004"

In [24]:
treated = ee.Image(treated_path)
baseline_fm40 = ee.Image("projects/pyregence-ee/assets/conus/landfire/fbfm40/LF2022_FBFM40_220")

In [25]:
# test ee blob table encoding working properly
blob = ee.Blob("gs://landfire/LFTFCT_tables/million-acres-LUTs/LANDFIRE fuel model subs - FM40.csv")
table = parse_txt(blob)

# read in the encoded value list as numeric
from_codes = to_numeric(ee.List(table.get("Original FM40_number")))
# read the list of values to remap to as numeric
to_codes = to_numeric(ee.List(table.get("Updated FM40_number")))

print(from_codes.getInfo())
print(to_codes.getInfo())

[91, 92, 93, 98, 99, 101, 102, 103, 104, 121, 122, 123, 141, 142, 143, 144, 145, 146, 147, 161, 162, 163, 165, 181, 182, 183, 184, 185, 186, 187, 188, 189, 201, 202]
[91, 92, 93, 98, 99, 101, 101, 101, 101, 121, 121, 121, 141, 141, 141, 141, 142, 141, 142, 161, 161, 161, 161, 181, 181, 181, 181, 181, 181, 181, 181, 181, 201, 201]


For 2.3M we need each run Year FM40 to represent what that FM40 value would be at the end so for run1 (Year 5) it would just be pixel values 1 we are updating FM40s to

For run2 (Year10) it would be pixel values 1.5 and 2 (areas treated in YI1 and YI2, as well as areas only treated in YI2) ? 

For run3 (Year20) it would be pixel values 3.5 and 4?

In [ ]:
# do this with a little less complexity since the 2.3M SC/CC situation is a little confusing
# double-chekc this doens't change for WUI?
treatment_img = ee.Image("") # once it exports

year5_treated = treatment_img.eq(1).selfMask()
year10_treated = treatment_img.eq(1.5).Or(treatment_img.eq(2)).selfMask()
year20_treated = treatment_img.eq(3.5).Or(treatment_img.eq(4)).selfMask()

year5_remap = baseline_fm40.updateMask(year5_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
year5_output = year5_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',1)

year10_remap = baseline_fm40.updateMask(year10_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
year10_output = year10_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',2)

year20_remap = baseline_fm40.updateMask(year20_treated).remap(from_codes,to_codes) # remap FM40 in specific treated areas
year20_output = year20_remap.unmask(baseline_fm40).rename('new_fm40').set('GFyearInterval',3)

updated_fm40s = ee.List([year5_output,
                         year10_output,
                         year20_output])

check_one = ee.Image(ee.List(updated_fm40s).get(1)) # second Fm40



In [26]:
# Keep in mind for WUI scenarios, ee_treatments() images don't contain pixel value 1 (YI1) because YI1 and YI4 share same footprint, 
# We re-treated top 25th pctile (originally YI1) in YI4 and the mosaicking burned over the 1's with 4s
# if 'WUI' in treated_path:
#     ee_treat_YI = ee.List([4,2,4])
# else:
#     ee_treat_YI = ee.List([1,2,4])

# gridfire_YI = ee.List([1,2,3])
# zipped = ee_treat_YI.zip(gridfire_YI)
# # print(zipped.getInfo())
# def per_year_update(z):
#     pixel_value = ee.List(z).get(0)
#     gridfire_yi = ee.List(z).get(1)
#     treated_mask = treated.eq(ee.Number(pixel_value)).selfMask()
#     updated_fm40 = baseline_fm40.updateMask(treated_mask).remap(from_codes,to_codes) # remap FM40 in specific treated areas
#     output = updated_fm40.unmask(baseline_fm40) # fill in updated fm40 with baseline in non-treated areas
#     return output.rename('new_fm40').set('GFyearInterval',gridfire_yi)

# updated_fm40s = zipped.map(per_year_update)

# check_one = ee.Image(ee.List(updated_fm40s).get(1)) # second Fm40
# print(check_one.get('GFyearInterval').getInfo())

In [27]:
# check that remap is working inside treated boxes, 
# should see 165s go to 161s, etc.. see above from_codes and to_codes printed out

# import geemap
# Map = geemap.Map()

# Map.addLayer(baseline_fm40,{'min':90,'max':205},'baseline')
# Map.addLayer(treated.eq(2).selfMask(),{min:0,max:1},'treated Y1') # treated mask for second FM40
# Map.addLayer(check_one,{'min':90,'max':205},'updated Y1')

# Map.centerObject(treated,11)

# Map

In [28]:
# Export each FM40 in list
# use California as output region
ca = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq("NAME","California")).first().geometry()

# use RUNID string as sub-folder in output Gdrive folder
runID_drive = os.path.basename(os.path.dirname(treated_path)) # to get RUNID string

for img in list(range(updated_fm40s.size().getInfo())):
    output = ee.Image(updated_fm40s.get(img)).toByte()
    year_interval = output.get('GFyearInterval').getInfo()
    desc = f"UpdateFM40_GFrun{year_interval}"
    output_path = os.path.dirname(treated_path)+"/"+f"FM40_GFrun{year_interval}"
    ee_treatments.export_img(output,desc,output_path,ca,30,'EPSG:3310')
    
    ee_treatments.export_image_to_drive(image=output,
                                        description=desc+'-Drive',
                                        region=ca,
                                        scale=30,
                                        crs='EPSG:3310',
                                        folder=f"MAS_FM40_for_GridFire",
                                        fileNamePrefix=f"{runID_drive}_FM40_GFrun{year_interval}",
                                        maxPixels=1e12)

Export Started for projects/mas-gee/assets/treatment_scenarios/RunID33/FM40_GFrun1
Export Started (Drive): RunID33_FM40_GFrun1
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID33/FM40_GFrun2
Export Started (Drive): RunID33_FM40_GFrun2
Export Started for projects/mas-gee/assets/treatment_scenarios/RunID33/FM40_GFrun3
Export Started (Drive): RunID33_FM40_GFrun3
